In [6]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, FunctionTransformer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, balanced_accuracy_score

In [3]:
df = pd.read_csv('../data/df_train.csv')
df_test = pd.read_csv('../data/df_test.csv')
df['ShapeFactor5'] = df['MajorAxisLength'] / df['Perimeter']
df_test['ShapeFactor5'] = df_test['MajorAxisLength'] / df_test['Perimeter']

# Funkcje 

In [4]:
def replace_outliers(data):
    for column in data.columns:
        if data[column].dtype in ['int64', 'float64']:  # tylko dla kolumn numerycznych
            Q1 = data.groupby('Class')[column].quantile(0.25)
            Q3 = data.groupby('Class')[column].quantile(0.75)
            IQR = Q3 - Q1

            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            mean = data.groupby('Class')[column].mean()
            for class_name in data['Class'].unique():
                data.loc[(data['Class'] == class_name) & (data[column] < lower_bound[class_name]), column] = mean[class_name]
                data.loc[(data['Class'] == class_name) & (data[column] > upper_bound[class_name]), column] = mean[class_name]
    return data

### cośtam

In [7]:
outlier_replacer = FunctionTransformer(func=replace_outliers)

pipeline = Pipeline(steps=[
    ('outlier_replacer', outlier_replacer)
])